In [449]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import seaborn as sns
from scipy import stats

In [450]:
Data=pd.read_csv('/Users/Supriya/Desktop/diabetes.csv')

In [451]:
Data.head()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1


In [452]:
Data[['Glucose','BloodPressure','SkinThickness','Insulin','BMI']] = Data[['Glucose','BloodPressure','SkinThickness','Insulin','BMI']].replace(0,np.NaN)

In [453]:
print(Data.isnull().sum())

Pregnancies                   0
Glucose                       5
BloodPressure                35
SkinThickness               227
Insulin                     374
BMI                          11
DiabetesPedigreeFunction      0
Age                           0
Outcome                       0
dtype: int64


In [454]:
Data['Glucose'].fillna(Data['Glucose'].mean(), inplace = True)
Data['BloodPressure'].fillna(Data['BloodPressure'].mean(), inplace = True)
Data['SkinThickness'].fillna(Data['SkinThickness'].median(), inplace = True)
Data['Insulin'].fillna(Data['Insulin'].median(), inplace = True)
Data['BMI'].fillna(Data['BMI'].median(), inplace = True)

In [455]:
print(Data.isnull().sum())

Pregnancies                 0
Glucose                     0
BloodPressure               0
SkinThickness               0
Insulin                     0
BMI                         0
DiabetesPedigreeFunction    0
Age                         0
Outcome                     0
dtype: int64


In [456]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X =  pd.DataFrame(sc.fit_transform(Data.drop(["Outcome"],axis = 1),),
        columns=['Pregnancies', 'Glucose', 'BloodPressure', 'SkinThickness', 'Insulin',
       'BMI', 'DiabetesPedigreeFunction', 'Age'])

In [457]:
X.head()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age
0,0.639947,0.865108,-0.033518,0.670643,-0.181541,0.166619,0.468492,1.425995
1,-0.844885,-1.206162,-0.529859,-0.012301,-0.181541,-0.852200,-0.365061,-0.190672
2,1.233880,2.015813,-0.695306,-0.012301,-0.181541,-1.332500,0.604397,-0.105584
3,-0.844885,-1.074652,-0.529859,-0.695245,-0.540642,-0.633881,-0.920763,-1.041549
4,-1.141852,0.503458,-2.680669,0.670643,0.316566,1.549303,5.484909,-0.020496


In [458]:
Y = Data['Outcome']

In [459]:
X_train,X_test,Y_train,Y_test = train_test_split(X,Y,test_size = 0.2,random_state = 0)

In [460]:
X_train.shape,X_test.shape,Y_train.shape,Y_test.shape

((614, 8), (154, 8), (614,), (154,))

In [461]:
#Euclidean Distance
def E_Distance(x1, x2):
    distance = np.square(x1 - x2)
    return np.sqrt(distance.sum(axis=1))

# K-NN model
def knn(trainingSet_x, testInstance,trainingset_y, k):
    dist = E_Distance(trainingSet_x,testInstance)
    distances={i:dist.iloc[i] for i in range(len(dist))}
    sortdist = sorted(distances.items(), key=operator.itemgetter(1))
    row_index=[sortdist[i][0] for i in range(k)]
    y_temp=stats.mode(trainingset_y.iloc[row_index].values)
    return(y_temp[0][0]) 

# supplying test data to the model
k=11
y_pred=[]
for i in range(len(X_test)):
    y_pred.append(knn(X_train, X_test.iloc[i],Y_train, k))
print(len(y_pred))


154


In [462]:
#Confusion Matrix
tp=len([i for i in range(Y_test.shape[0]) if Y_test.iloc[i]==1 and y_pred[i]==1])
tn=len([i for i in range(Y_test.shape[0]) if Y_test.iloc[i]==0 and y_pred[i]==0])
fp=len([i for i in range(Y_test.shape[0]) if Y_test.iloc[i]==0 and y_pred[i]==1])
fn=len([i for i in range(Y_test.shape[0]) if Y_test.iloc[i]==1 and y_pred[i]==0])
confusion_matrix=np.array([[tp,fn],[fp,tn]])
print(confusion_matrix,type(confusion_matrix))

[[31 16]
 [ 9 98]] <class 'numpy.ndarray'>


In [463]:
tp=confusion_matrix[0,0]
fp=confusion_matrix[1,0]
tn=confusion_matrix[1,1]
fn=confusion_matrix[0,1]

Accuracy=(tp+tn)/(confusion_matrix.sum())
Recall=tp/(tp+fn)
Precision=tp/(tp+fp)
F=(2*Recall*Precision)/(Recall+Precision)

print('Accuracy= %.2f'%Accuracy,'\nRecall= %.2f'%Recall,'\nPrecision= %.2f'%Precision,'\nF= %.2f'%F)

Accuracy= 0.84 
Recall= 0.66 
Precision= 0.78 
F= 0.71
